<a href="https://colab.research.google.com/github/gisa-ceinfo-sms-sp/isacapitalsp/blob/main/ISA_CAPITAL_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###CODIGO-BASE ISA CAPITAL 2023 SMS-SP V.26052025_16h21m###
###ELABORAÇÂO: José Olimpio Moura de Albuquerque (NADEpi/CIEVS/DEVE/COVISA/SEABEVS), Marcelo Antunes Failla (GISA/CEInfo/CIS/SERMAP)###

#INSTRUÇÕES PARA RODAR O SCRIPT EM SUA MÁQUINA (PROCESSAMENTO LOCAL)#
#1. CRIE A PASTA "c:/ISA_Capital"#
#2. Insira o arquivo "ISACapital17042025TreinamentoComPesosExcluiVar.SAV"#
#3. Copie este código para o seu R Studio em sua máquina, em um novo script ou no script já em uso por versões anteriores#
#4. Selecione todas as linhas e clique em "run"#
#5. Os arquivos de saída serão salvos na mesma pasta onde está a base "c:/ISA_Capital"#

#INSTRUÇÕES PARA RODAR O SCRIPT VIA GOOGLE COLAB#
#1. Faça upload do arquivo "ISACapital17042025TreinamentoComPesosExcluiVar.SAV"#
#2. Selecione todas as linhas e clique no "botão de play" ou em "ambiente de execução>executar seleção"#
#3. Os arquivos de saída salvos na pasta "/content"#


install.packages("haven")   # Para ler arquivos .sav
install.packages("survey")  # O pacote survey realiza a análise de amostras complexas
install.packages("srvyr")  # O pacote srvyr baseado no 'survey' realiza a análise no padrão do tidyverse
install.packages("labelled")
install.packages("flextable")
install.packages("officer")

library(tidyverse)
library(haven)
library(survey)
library(srvyr)
library(purrr)
library(tibble)
library(labelled)
library(flextable)
library(officer)
library(glue)


#PROCESSAMENTO LOCAL - define a pasta onde estão os arquivos (remova o "#" da linha abaixo)
#setwd("c:/ISA_Capital")

#PROCESSAMENTO ONLINE - certifique-se da realização do upload do arquivo "ISACapital17042025TreinamentoComPesosExcluiVar.SAV"

#ler o arquivo de dados do SPSS
dados <- read_sav("/ISACapital17042025TreinamentoComPesosExcluiVar.SAV")



#exclui os registros com o peso final em branco
dados2 <- dados %>%
  mutate(
    FX_ETARIA = cut(
      IDADECALC,
      breaks = c(10, 20, 30, 40, 50, 60, 70, 80, 90, Inf),
      right = FALSE,
      labels = c(
        "10-19", "20-29", "30-39", "40-49",
        "50-59", "60-69", "70-79", "80-89", "90 ou +"
      )
    )
  ) %>%
  filter(!is.na(pesofinal))




## gera um data frame com os nomes das variáveis
# Função segura para extrair 1º label (ou NA se não houver)
extrair_label <- function(x) {
  lbl <- attr(x, "label")
  if (is.null(lbl)) return(NA_character_)
  if (length(lbl) > 1) return(paste(lbl, collapse = " | "))
  return(as.character(lbl))
}

# Função segura para extrair val_labels
extrair_val_labels <- function(x) {
  vl <- val_labels(x)
  if (is.null(vl)) return(NA_character_)
  paste(names(vl), "=", vl, collapse = "; ")
}

# Aplica ao banco
variaveis_info <- tibble(
  nome_variavel   = names(dados),
  label_variavel  = map_chr(dados, extrair_label),
  categorias      = map_chr(dados, extrair_val_labels),
  tipo_variavel   = map_chr(dados, ~ class(.)[1])
)


### Gera o arquivo CSV com as informações das variáveis
write.csv2(variaveis_info, "dicionario_variaveis.csv", row.names = FALSE, fileEncoding = "Windows-1252")


################################################################

# Define o desenho do estudo
des <- svydesign(
  id = ~cod_setor,
  strata = ~cod_coord,
  weights = ~pesofinal,
  data = dados2,
  nest = TRUE
)

# Cria versão srvyr do design
des_srvyr <- as_survey_design(des)

###################################################################################
########################## FAZ A TABULAÇÃO DAS VARIÁVEIS ##########################

# Exemplo com sexo
# o comando 'mutate(sexo = as_factor(b08_sexo))' recupera a variável b08_sexo do banco de dados
# e transforma em fator com a descrição do código que já está no arquivo .sav


freq_sexo <- des_srvyr %>%
  filter(!is.na(b07_sexo_geral)) %>%
  mutate(sexo = as_factor(b07_sexo_geral)) %>%
  group_by(sexo) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)


freq_morbidade2semanas <- des_srvyr %>%
  filter(!is.na(c101_morbidade2semanas)) %>%
  mutate(morbidade2semanas = as_factor(c101_morbidade2semanas)) %>%
  group_by(morbidade2semanas) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)

freq_hipertensao <- des_srvyr %>%
  filter(!is.na(c201a_hipertensao)) %>%
  mutate(hipertensao = as_factor(c201a_hipertensao)) %>%
  group_by(hipertensao) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)

freq_diabetes <- des_srvyr %>%
  filter(!is.na(c202a_diabetes)) %>%
  mutate(diabetes = as_factor(c202a_diabetes)) %>%
  group_by(diabetes) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)

freq_infarto <- des_srvyr %>%
  filter(!is.na(c203a_infarto)) %>%
  mutate(infarto = as_factor(c203a_infarto)) %>%
  group_by(infarto) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)

freq_avc <- des_srvyr %>%
  filter(!is.na(c205a_avc)) %>%
  mutate(avc = as_factor(c205a_avc)) %>%
  group_by(avc) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)


freq_cancer <- des_srvyr %>%
  filter(!is.na(c206a_cancer)) %>%
  mutate(cancer = as_factor(c206a_cancer)) %>%
  group_by(cancer) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)


freq_fx_etaria <- des_srvyr %>%
  filter(!is.na(FX_ETARIA)) %>%
  mutate(hipertensao = as_factor(FX_ETARIA)) %>%
  group_by(FX_ETARIA) %>%
  summarise(prop = survey_mean(vartype = "ci", proportion = TRUE)) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(Proporção = prop, IC_inf = prop_low, IC_sup = prop_upp)


#### GERAR DOCUMENTO COM AS TABELAS DE FREQUÊNCIA


# 🧩 Função robusta para gerar flextable
formatar_tabela <- function(df, titulo) {
  if (!is.data.frame(df) || nrow(df) == 0) {
    warning(paste("Tabela inválida ou vazia:", titulo))
    return(flextable(data.frame(Mensagem = "Tabela não disponível")))
  }
  df <- as.data.frame(df)  # garante compatibilidade
  flextable(df) %>%
    set_caption(caption = titulo) %>%
    autofit() %>%
    set_table_properties(width = 0.66, layout = "autofit")  # ⇐ Reduz largura para 66%
}

# 📄 Cria o documento Word
doc <- read_docx() %>%
  body_add_par("Tabelas de Frequência do Inquérito de Saúde", style = "heading 1") %>%
  body_add_flextable(formatar_tabela(freq_sexo, "Tabela 1 - Frequência por Sexo")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_morbidade2semanas, "Tabela 2 - Morbidade nas Últimas 2 Semanas")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_hipertensao, "Tabela 4 - Hipertensão")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_diabetes, "Tabela 5 - Diabetes")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_infarto, "Tabela 6 - Infarto")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_avc, "Tabela 7 - AVC")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_cancer, "Tabela 8 - Câncer")) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(formatar_tabela(freq_fx_etaria, "Tabela 9 - Frequência por Faixa Etária"))

# 💾 Salva o documento
print(doc, target = "frequencias_ISA_Capital.docx")


#################################################################
################## TABELAS CRUZADAS #############################
################################################################

# Cria tabela cruzada de sexo x morbidade 2 semanas
tab_cross_sexo_morbidade2semanas <- des_srvyr %>%
  filter(!is.na(b07_sexo_geral)) %>%
  filter(!is.na(c101_morbidade2semanas)) %>%
  mutate(
    sexo = as_factor(b07_sexo_geral),
    morbidade2semanas = as_factor(c101_morbidade2semanas)
  ) %>%
  group_by(sexo, morbidade2semanas) %>%
  summarise(
    prop = survey_mean(proportion = TRUE, vartype = "ci"),
    .groups = "drop"
  ) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(
    Proporção = prop,
    IC_inf = prop_low,
    IC_sup = prop_upp
  )


# Formatar tabela no padrão do STATA
tab_cruzada_formatada_sexo_morbidade2semanas <- tab_cross_sexo_morbidade2semanas %>%
  mutate(
    categoria = glue("{round(Proporção, 1)}% [{round(IC_inf, 1)}–{round(IC_sup, 1)}]")
  ) %>%
  select(sexo, morbidade2semanas, categoria) %>%
  pivot_wider(
    names_from = morbidade2semanas,
    values_from = categoria
  )

## Formatar a tabela para o flextable
flextable(tab_cruzada_formatada_sexo_morbidade2semanas) %>%
  set_caption("Tabela cruzada: Sexo x Morbidade nas últimas 2 semanas") %>%
  autofit()



# Cria tabela cruzada de hipertensão x faixa etaria
tab_cross_hipertensao_faixa_etaria <- des_srvyr %>%
  filter(!is.na(FX_ETARIA)) %>%
  filter(!is.na(c201a_hipertensao)) %>%
  mutate(
    hipertensao = as_factor(c201a_hipertensao)
  ) %>%
  group_by(FX_ETARIA, hipertensao) %>%
  summarise(
    prop = survey_mean(proportion = TRUE, vartype = "ci"),
    .groups = "drop"
  ) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(
    Proporção = prop,
    IC_inf = prop_low,
    IC_sup = prop_upp
  )

# Formatar tabela no padrão do STATA
tab_cruzada_formatada_hipertensao_faixa_etaria <- tab_cross_hipertensao_faixa_etaria %>%
  mutate(
    categoria = glue("{round(Proporção, 1)}% [{round(IC_inf, 1)}–{round(IC_sup, 1)}]")
  ) %>%
  select(hipertensao, FX_ETARIA, categoria) %>%
  pivot_wider(
    names_from = hipertensao,
    values_from = categoria
  )

## Formatar a tabela para o flextable
flextable(tab_cruzada_formatada_hipertensao_faixa_etaria) %>%
  set_caption("Tabela cruzada: Hipertensão x Faixa Etária") %>%
  autofit()



# Cria tabela cruzada de infarto x faixa etaria
tab_cross_infarto_faixa_etaria <- des_srvyr %>%
  filter(!is.na(FX_ETARIA)) %>%
  filter(!is.na(c203a_infarto)) %>%
  mutate(
    infarto = as_factor(c203a_infarto)
  ) %>%
  group_by(FX_ETARIA, infarto) %>%
  summarise(
    prop = survey_mean(proportion = TRUE, vartype = "ci"),
    .groups = "drop"
  ) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(
    Proporção = prop,
    IC_inf = prop_low,
    IC_sup = prop_upp
  )

# Formatar tabela no padrão do STATA
tab_cruzada_formatada_infarto_faixa_etaria <- tab_cross_infarto_faixa_etaria %>%
  mutate(
    categoria = glue("{round(Proporção, 1)}% [{round(IC_inf, 1)}–{round(IC_sup, 1)}]")
  ) %>%
  select(infarto, FX_ETARIA, categoria) %>%
  pivot_wider(
    names_from = infarto,
    values_from = categoria
  )

## Formatar a tabela para o flextable
flextable(tab_cruzada_formatada_infarto_faixa_etaria) %>%
  set_caption("Tabela cruzada: Infarto x Faixa Etária") %>%
  autofit()

##
# Cria tabela cruzada de CRS x morbidade 2 semanas
tab_cross_CRS_morbidade2semanas <- des_srvyr %>%
  filter(!is.na(cod_coord)) %>%
  filter(!is.na(c101_morbidade2semanas)) %>%
  mutate(
    CRS = as_factor(cod_coord),
    morbidade2semanas = as_factor(c101_morbidade2semanas)
  ) %>%
  group_by(CRS, morbidade2semanas) %>%
  summarise(
    prop = survey_mean(proportion = TRUE, vartype = "ci"),
    .groups = "drop"
  ) %>%
  mutate(across(c(prop, prop_low, prop_upp), ~ . * 100)) %>%
  rename(
    Proporção = prop,
    IC_inf = prop_low,
    IC_sup = prop_upp
  )


# Formatar tabela no padrão do STATA
tab_cruzada_formatada_CRS_morbidade2semanas <- tab_cross_CRS_morbidade2semanas %>%
  mutate(
    categoria = glue("{round(Proporção, 1)}% [{round(IC_inf, 1)}–{round(IC_sup, 1)}]")
  ) %>%
  select(CRS, morbidade2semanas, categoria) %>%
  pivot_wider(
    names_from = morbidade2semanas,
    values_from = categoria
  )

## Formatar a tabela para o flextable
flextable(tab_cruzada_formatada_CRS_morbidade2semanas) %>%
  set_caption("Tabela cruzada: CRS x Morbidade nas últimas 2 semanas") %>%
  autofit()



## Salvar no arquivo .DOCX
# 1. Criar flextables com legenda
ftx1 <- flextable(tab_cruzada_formatada_sexo_morbidade2semanas) %>%
  set_caption("Tabela 1. Sexo x Morbidade nas últimas 2 semanas") %>%
  autofit()

ftx2 <- flextable(tab_cruzada_formatada_hipertensao_faixa_etaria) %>%
  set_caption("Tabela 2. Hipertensão x Faixa Etária") %>%
  autofit()

ftx3 <- flextable(tab_cruzada_formatada_infarto_faixa_etaria) %>%
  set_caption("Tabela 3. Infarto x Faixa Etária") %>%
  autofit()

ftx4 <- flextable(tab_cruzada_formatada_CRS_morbidade2semanas) %>%
  set_caption("Tabela 4. CRS x Morbidade nas últimas 2 semanas") %>%
  autofit()

# 2. Criar documento Word
doc <- read_docx()

# 3. Adicionar as tabelas ao documento
doc <- doc %>%
  body_add_par("Tabelas Cruzadas do ISA Capital", style = "heading 1") %>%
  body_add_flextable(ftx1) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(ftx2) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(ftx3) %>%
  body_add_par("", style = "Normal") %>%
  body_add_flextable(ftx4)

# 4. Salvar o documento
print(doc, target = "tabelas_cruzadas_ISA_Capital.docx")
# Fim do script




a flextable object.
col_keys: `sexo`, `Não`, `Sim`, `NS/NR` 
header has 1 row(s) 
body has 3 row(s) 
original dataset sample: 
       sexo               Não               Sim        NS/NR
1 Masculino   78.5% [75.7–81] 21.4% [18.9–24.1] 0.1% [0–0.4]
2  Feminino 71.5% [68.8–74.1]   28.3% [25.7–31] 0.1% [0–0.4]
3     Outro 83.1% [23.1–98.8]  16.9% [1.2–76.9]         <NA>

a flextable object.
col_keys: `FX_ETARIA`, `Não`, `Sim` 
header has 1 row(s) 
body has 9 row(s) 
original dataset sample: 
  FX_ETARIA               Não               Sim
1     10-19 99.3% [98.8–99.6]    0.7% [0.4–1.2]
2     20-29   97.1% [95–98.3]      2.9% [1.7–5]
3     30-39 90.6% [87.3–93.1]   9.4% [6.9–12.7]
4     40-49 80.4% [76.6–83.7] 19.6% [16.3–23.4]
5     50-59   61% [56.2–65.6]   39% [34.4–43.8]

a flextable object.
col_keys: `FX_ETARIA`, `Não`, `NS/NR`, `Sim` 
header has 1 row(s) 
body has 9 row(s) 
original dataset sample: 
  FX_ETARIA               Não        NS/NR            Sim
1     10-19  99.9% [99.3–100] 0.1% [0–0.7]           <NA>
2     20-29  99.8% [99.2–100]         <NA>   0.2% [0–0.8]
3     30-39 99.6% [98.7–99.9]         <NA> 0.4% [0.1–1.3]
4     40-49 99.1% [97.8–99.6]   0.3% [0–2] 0.7% [0.3–1.6]
5     50-59 97.6% [96.2–98.5] 0.2% [0–1.4] 2.2% [1.3–3.6]

a flextable object.
col_keys: `CRS`, `Não`, `Sim`, `NS/NR` 
header has 1 row(s) 
body has 6 row(s) 
original dataset sample: 
  CRS               Não               Sim          NS/NR
1   1   73.6% [69–77.7]   26.4% [22.3–31]           <NA>
2   2 73.1% [67.5–78.1] 26.8% [21.8–32.4]   0.1% [0–0.6]
3   3 74.2% [69.5–78.5] 25.4% [21.1–30.2] 0.3% [0.1–1.3]
4   4 78.5% [74.6–81.9]   21.4% [18–25.3]   0.1% [0–0.7]
5   5 74.5% [70.3–78.2] 25.5% [21.8–29.7]           <NA>